In [1]:
from fairsearchcore.models import FairScoreDoc
import fairsearchcore as fsc
import pandas as pd
import numpy as np
import os
from copy import copy

In [2]:
def include_is_protected(df, protected_col: str, protected_values: list, output_col: str):
    df[output_col] = np.where(df[protected_col].isin(protected_values), True, False)
    return df


def generate_ranking(df, position_col: str, is_protected_col: str):
    ranking = []

    for index, row in df.iterrows():
        ranking.append(FairScoreDoc(row[position_col], row[position_col], row[is_protected_col]))

    return ranking


def is_fair_ranking(ranking, k=20, p=0.25, alpha=0.1):
    fair = fsc.Fair(k, p, alpha)
    return fair.is_fair(ranking)


def generate_p_values(start_value=0.5, end_value=0.95, distance=0.5):
    return list(np.arange(start_value * 100, end_value * 100, distance * 100) / 100)


def compute_min_p_fair(ranking, p_values, k=50, alpha=0.1):
    fair_results = {}
    for p_value in p_values:
        fair_results[p_value] = is_fair_ranking(ranking, k=k, p=p_value, alpha=alpha)

    max_p = 0

    for item in fair_results.items():
        if item[1] is True:
            max_p = item[0]
        else:
            break

    return max_p, fair_results

In [3]:
platforms = ['italki', 'preply', 'verbling']
results = pd.DataFrame(columns=['language', 'platform', 'k', 'max_p', 'original_proportion', 'alpha'])
protected_values = ['female']
protected_column = 'gender_tuned'

for platform in platforms:
    path = '../../data/fair/{}.csv'.format(platform)
    df = pd.read_csv(path, index_col=0)
    df = include_is_protected(df, protected_column, protected_values, "is_gender_protected")
    languages = list(df.language.unique())
    for language in languages:
        try:
            print("{} - {}".format(platform, language))
            temp = copy(df[df['language']==language])

            ranking = generate_ranking(temp, 'position', 'is_gender_protected')
            
            proportion = temp.is_gender_protected.value_counts()[True]/len(temp)

            p_values = generate_p_values(0.10, 1, 0.1)

            if len(df) < 5:
                ks = [len(temp)]
            else:
                ks = [3, 5] + [i for i in range(10, min(len(temp), 99)+1, 30)] + [min(100, len(temp))]
                if len(ks)==2: # If df is shorter than 10, we include the whole dataframe
                    ks.append(len(temp))

            for k in ks:
                alpha = 0.1
                max_p, fair_results = compute_min_p_fair(ranking[:k], p_values, k=k, alpha=alpha)
                results = results.append({'language': language, 'platform': platform, 'k': k, 'max_p': max_p, 'original_proportion': proportion, 'alpha': alpha}, ignore_index=True)
        except:
            print("Error in {} - {}".format(platform, language))

results.to_csv('../data/results/gender_fair_analysis.csv')

italki - Thai


/Users/javi/miniconda3/envs/tfg/lib/python3.8/site-packages/fairsearchcore/fair.py:165: UserWarning: Library has not been tested with values outside this range
  warnings.warn("Library has not been tested with values outside this range")


italki - German
italki - Portuguese
italki - Hebrew
italki - Hindi
italki - Romanian
italki - Chinese (Mandarin)
italki - Dutch
italki - Indonesian
italki - Italian
italki - Russian
italki - Greek
italki - English
italki - Polish
italki - Arabic
italki - Turkish
italki - Vietnamese
italki - Japanese
italki - Spanish
italki - French
italki - Korean
italki - Persian (Farsi)
preply - Thai
preply - German
preply - Portuguese
preply - Hebrew
preply - Hindi
preply - Romanian
preply - Persian
preply - Dutch
preply - Indonesian
preply - Italian
preply - Russian
preply - Greek
preply - English
preply - Polish
preply - Arabic
preply - Turkish
preply - Vietnamese
preply - Chinese
preply - Japanese
preply - Spanish
preply - French
preply - Korean
verbling - Thai
verbling - German
verbling - Portuguese
verbling - Hebrew
verbling - Hindi
verbling - Romanian
verbling - Persian
verbling - Dutch
verbling - Indonesian
verbling - Italian
verbling - Russian
verbling - Greek
verbling - English
verbling - P

In [4]:
results

,language,platform,k,max_p,original_proportion,alpha
0,Thai,italki,3,0.8,0.658228,0.1
1,Thai,italki,5,0.8,0.658228,0.1
2,Thai,italki,10,0.7,0.658228,0.1
3,Thai,italki,40,0.7,0.658228,0.1
4,Thai,italki,70,0.7,0.658228,0.1
...,...,...,...,...,...,...
356,Korean,verbling,3,0.9,0.595745,0.1
357,Korean,verbling,5,0.9,0.595745,0.1
358,Korean,verbling,10,0.8,0.595745,0.1
359,Korean,verbling,40,0.7,0.595745,0.1


In [5]:
results.head(30)

,language,platform,k,max_p,original_proportion,alpha
0,Thai,italki,3,0.8,0.658228,0.1
1,Thai,italki,5,0.8,0.658228,0.1
2,Thai,italki,10,0.7,0.658228,0.1
3,Thai,italki,40,0.7,0.658228,0.1
4,Thai,italki,70,0.7,0.658228,0.1
5,Thai,italki,79,0.7,0.658228,0.1
6,German,italki,3,0.6,0.557500,0.1
7,German,italki,5,0.6,0.557500,0.1
8,German,italki,10,0.3,0.557500,0.1
9,German,italki,40,0.3,0.557500,0.1


In [6]:
results[results['max_p']<results['original_proportion']].head(50)

,language,platform,k,max_p,original_proportion,alpha
8,German,italki,10,0.3,0.557500,0.1
9,German,italki,40,0.3,0.557500,0.1
10,German,italki,70,0.3,0.557500,0.1
11,German,italki,100,0.3,0.557500,0.1
13,Portuguese,italki,5,0.4,0.508523,0.1
14,Portuguese,italki,10,0.4,0.508523,0.1
15,Portuguese,italki,40,0.5,0.508523,0.1
16,Portuguese,italki,70,0.5,0.508523,0.1
17,Portuguese,italki,100,0.5,0.508523,0.1
20,Hebrew,italki,10,0.4,0.415385,0.1
